In [33]:
import pandas as pd
import networkx as nx
import pickle
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

In [18]:
tissue_samples = pd.read_csv('GTEx_v7_Annotations_SampleAttributesDS.txt', sep = '\t')
tissue_list = np.sort(tissue_samples['SMTSD'].unique())
tissue_list

array(['Adipose - Subcutaneous', 'Adipose - Visceral (Omentum)',
       'Adrenal Gland', 'Artery - Aorta', 'Artery - Coronary',
       'Artery - Tibial', 'Bladder', 'Brain - Amygdala',
       'Brain - Anterior cingulate cortex (BA24)',
       'Brain - Caudate (basal ganglia)', 'Brain - Cerebellar Hemisphere',
       'Brain - Cerebellum', 'Brain - Cortex',
       'Brain - Frontal Cortex (BA9)', 'Brain - Hippocampus',
       'Brain - Hypothalamus',
       'Brain - Nucleus accumbens (basal ganglia)',
       'Brain - Putamen (basal ganglia)',
       'Brain - Spinal cord (cervical c-1)', 'Brain - Substantia nigra',
       'Breast - Mammary Tissue', 'Cells - EBV-transformed lymphocytes',
       'Cells - Leukemia cell line (CML)',
       'Cells - Transformed fibroblasts', 'Cervix - Ectocervix',
       'Cervix - Endocervix', 'Colon - Sigmoid', 'Colon - Transverse',
       'Esophagus - Gastroesophageal Junction', 'Esophagus - Mucosa',
       'Esophagus - Muscularis', 'Fallopian Tube',
       'H

In [20]:
tissue_samples = pd.read_csv('GTEx_v7_Annotations_SampleAttributesDS.txt', sep = '\t')
tissue_list = tissue_samples.loc[tissue_samples['SMTSD'] == 'Skin - Sun Exposed (Lower leg)']['SAMPID'].to_numpy()
len(tissue_list)

592

In [21]:
columns_data = pd.read_csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct', index_col = 'Name', sep = '\t', header = 2 , nrows = 2)
available_cols = columns_data.columns
common_columns = list(set(available_cols).intersection(tissue_list))
#common_columns.append('Name')
print(len(common_columns))

473


In [22]:
variance_dataframe = pd.read_csv('variance_dataset.csv')
variance_dataframe.index.values

array([    0,     1,     2, ..., 56199, 56200, 56201])

In [23]:
all_rows = pd.read_csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct', usecols = ['Name'], sep = '\t', header = 2)
len(all_rows)

56202

In [24]:
rows = variance_dataframe.nlargest(100, 'Variance').index.values
rows = np.append(rows, [0, 1, 2])
print(len(rows))
skip_rows = np.setdiff1d(all_rows.index.values,rows)
skip_rows

103


array([    3,     4,     5, ..., 56198, 56200, 56201])

In [25]:
def logic(index):
    if index in skip_rows:
        return True
    return False

small_dataset = pd.read_csv('GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct', usecols = common_columns, sep = '\t', header = 2 , skiprows= lambda x: logic(x))
small_dataset

,GTEX-111FC-0126-SM-5N9DL,GTEX-111VG-2426-SM-5GZXD,GTEX-1122O-2126-SM-5EGIR,GTEX-1128S-2326-SM-5GZZY,GTEX-113IC-0126-SM-5HL6T,GTEX-113JC-2326-SM-5EQ4E,GTEX-117XS-2726-SM-5N9BL,GTEX-117YW-2626-SM-5GZZH,GTEX-117YX-2326-SM-5H12W,GTEX-1192W-2626-SM-5Q5AF,...,GTEX-ZXG5-0126-SM-5GIEU,GTEX-ZY6K-1826-SM-5GZXK,GTEX-ZYFC-0226-SM-5NQ75,GTEX-ZYFD-0126-SM-5GIDL,GTEX-ZYFG-2326-SM-5E44B,GTEX-ZYT6-0226-SM-5NQ6T,GTEX-ZYW4-0126-SM-5E44A,GTEX-ZYY3-0126-SM-5GZY5,GTEX-ZZ64-1726-SM-5GZYB,GTEX-ZZPT-0226-SM-5E43X
0,0.09002,0.0000,0.1465,0.31420,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.0000,0.00000,0.30190,0.00000,0.0000,0.0000,0.13590,0.24360
1,0.00000,0.1032,0.1159,0.14500,0.1093,0.08478,0.02074,0.02543,0.17470,0.08499,...,0.05821,0.1211,0.3342,0.06508,0.25870,0.06844,0.1484,0.0320,0.02687,0.12040
2,0.07520,0.1634,0.0000,0.04375,0.0000,0.00000,0.04380,0.00000,0.05271,0.05984,...,0.00000,0.0000,0.0000,0.20620,0.08406,0.00000,0.0000,0.0676,0.11350,0.05088
3,66.00000,56.4800,54.2900,79.30000,3583.0000,61.59000,67.39000,57.83000,90.35000,43.30000,...,58.15000,356.1000,3644.0000,62.14000,101.90000,39.72000,46.2500,44.0900,338.50000,121.00000
4,0.00000,0.0000,0.0000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,0.17510,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,0.51110,0.7404,0.0000,0.00000,0.0000,0.00000,0.00000,0.73030,0.71650,0.00000,...,1.67100,0.0000,0.0000,1.86900,1.14300,0.00000,0.0000,1.8380,0.00000,2.76600
99,4574.00000,8746.0000,2226.0000,13840.00000,12440.0000,8464.00000,13200.00000,4802.00000,9458.00000,4116.00000,...,9742.00000,11630.0000,5442.0000,3748.00000,14190.00000,4662.00000,7964.0000,6532.0000,7734.00000,6332.00000
100,20330.00000,14810.0000,10180.0000,23940.00000,19960.0000,15940.00000,19600.00000,17870.00000,24220.00000,9106.00000,...,15740.00000,22900.0000,19100.0000,16040.00000,35570.00000,14990.00000,14820.0000,14350.0000,19220.00000,27760.00000
101,0.45690,0.0000,2.2310,2.12700,8.4190,2.90100,5.32200,1.30600,0.64050,0.72720,...,0.00000,0.6215,3.5740,2.92300,4.08600,0.00000,2.2860,4.1070,0.00000,0.00000


In [26]:
dataset = small_dataset

In [27]:
def get_bootstrapped_dataset():
    
    individuals = dataset.index.values
    bootstrapped_individuals = np.random.choice(individuals, len(individuals), replace=True)  #Get bootstrapped samples
    #print("bootstrapped_individuals")
    #print(bootstrapped_individuals)
    p_values = []
    corr_values = []

    #count = 0
    for i in bootstrapped_individuals:
        for j in bootstrapped_individuals:
        
            #count = count + 1
            ##if(count%20000 == 0):
            #    print(count)

            gene1 = dataset.loc[i].values
            gene2 = dataset.loc[j].values

            corr, p = spearmanr(gene1, gene2) #calculated spearman correlation for each pair

            corr_values.append(corr)
            p_values.append(p)

    results = multipletests(p_values, alpha = 0.05, method='fdr_bh')
    reject = results[0]
    p_values_corrected = results[1]
    #print("p value entries")
    #print(np.sum(np.array(p_values_corrected > 0.1)))
    G = {}

    for i in range(len(dataset)):
        temp = []
        for j in range(len(dataset)):
            if p_values_corrected[i*len(dataset) + j] > 0.1:  #If its greater than cutoff 
                temp.append(dataset.index[j])

        G.update({dataset.index[i] : temp})
    
    #print("graph array")
    #print(G)
    return G

In [28]:
def get_bit_vector():
    
    graph_dict = get_bootstrapped_dataset()
    
    graph_nodes = np.array(list(graph_dict.keys()))
    G = nx.Graph()

    for node in graph_nodes:
        connected_edges = np.array(graph_dict[node])

        for other_node in connected_edges:
            G.add_edge(node, other_node)    #Make networkx graph
    
    graph_nodes = list(G.nodes)  

    bit_vector = []
    for node1 in dataset.index.values:
        for node2 in dataset.index.values:
            if node1 != node2:
                if(G.has_edge(node1, node2)):  #If edge exists in coexpression network, make it 1
                    bit_vector.append(1)
                else:
                    bit_vector.append(0)    #If edge it does not exist in coexpression network, make it 0

    return bit_vector

In [29]:
def get_bit_matrix():
    
    B = 5000
    matrix = []
    for i in range(B):
        if(i % 200 == 0):
            print('B' + ' is ' + str(i))       #Get the binary string for B values and make the matrix
        bit_vector = get_bit_vector()
        #print('len of bit vector' + ' is ' + str(len(bit_vector)))
        matrix.append(bit_vector)
        
    return matrix

In [30]:
import time
start_time = time.time()
matrix = get_bit_matrix()
end_time = time.time()
print("Time spent " + str(end_time - start_time))

B is 0
B is 200
B is 400
B is 600
B is 800
B is 1000
B is 1200
B is 1400
B is 1600
B is 1800
B is 2000
B is 2200
B is 2400
B is 2600
B is 2800
B is 3000
B is 3200
B is 3400
B is 3600
B is 3800
B is 4000
B is 4200
B is 4400
B is 4600
B is 4800
Time spent 46705.57075381279


In [33]:
matrix.shape

AttributeError: 'list' object has no attribute 'shape'

In [31]:
matrix2 = np.array(matrix)
matrix2.shape

(5000, 10506)

In [32]:
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

matrix_array = asarray(matrix2)
savetxt('bootstrapped_data_Skin_Sun_Exposed.csv', matrix_array, delimiter=',')
matrix_array = loadtxt('bootstrapped_data_Skin_Sun_Exposed.csv', delimiter=',')
print(matrix_array.shape)

(5000, 10506)


In [37]:
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt

matrix2 = loadtxt('bootstrapped_data_muscleskeleton.csv', delimiter=',')
print(matrix2.shape)
savetxt('bootstrapped_data_muscleskeleton.csv', matrix2[:5000, :], delimiter=',')
matrix2 = loadtxt('bootstrapped_data_muscleskeleton.csv', delimiter=',')
print(matrix2.shape)

(6000, 10506)
(5000, 10506)


In [ ]:
B = 500
verify_G = nx.Graph()
conflict_G = nx.Graph()
matrix = matrix2[:500, ]
print(matrix.shape)
for i in range(matrix.shape[1]):
    
    if(i%1000 == 0):
        print(i)
    #temp = []
    for j in range(matrix.shape[1]):
        
        gamets = [0,0,0,0]
        for string in range(matrix.shape[0]):
            
            if(matrix[string][i] == 0 and matrix[string][j] == 1):
                gamets[1] = 1
            if(matrix[string][i] == 1 and matrix[string][j] == 0):
                gamets[2] = 1
            if(matrix[string][i] == 1 and matrix[string][j] == 1):
                gamets[3] = 1
                
            if(np.sum(gamets) == 3):
                conflict_G.add_edge(i, j)
                break
        


(500, 10506)
0
1000
2000


In [22]:
print(conflict_G.number_of_nodes())
print(conflict_G.number_of_edges())


2756
3795012


# Calculating cardinalty cover vertex

In [23]:
graph_nodes = list(conflict_G.nodes())

visited = {}
for node in graph_nodes:
    visited.update({node : False})


for u in graph_nodes:    
    #print(conflict_G.edges(u))
    if visited[u] == False:
        
        for v in conflict_G.edges(u):
            if visited[v[1]] == False:
                #print((u,v[1]), end = " ")
                visited[v[1]] = True
                visited[u] = True
                break
                

cardinality_vertex_cover = []
for node in graph_nodes:
    if visited[node]:
        cardinality_vertex_cover.append(node)
        
print(len(cardinality_vertex_cover))

2756
